# Data Science in Python == Data Science in Numpy <img src="logo.png",width=140,height=140, align="right">

The fundamental library for data science is numpy. __Numpy__ gives us *fast* and *powerful* tools for numerical operations on large, multi-dimensional arrays of data. Which as you can image is useful for much of data science!

Then, a kind soul called Wes McKinney decided to make our lives even easier. He created __pandas__, a library built on top of numpy which makes analysing messy, real-world datasets more intuitive. Pandas adds more functionality and a wonderfully useful 2-dimensional data structure known as a DataFrame.

Knowing how to use these libraries will make the slog of understanding your data and getting it into a useable state much easier. If you also understand _why_ these libraries matter, your code will be faster, more reuseable and better suited to being deployed into production. 

In [2]:
import numpy as np

## TL;DR : Check if numpy or pandas has a function that does it for you
If you don't read any further, that's the key lesson. Here's how you can do it:

In [ ]:
# give it a go
np.lookfor('love')

In [ ]:
# you could implement this for another library,
# such as pandas, with something like
[name for name in dir(pd) if 'max' in name]

In [ ]:
# think np has a sum method?
# let's check
np.su*?

In [ ]:
# how about pandas?
pd.s*?

In general, make extensive use of documentation & stackoverflow, and include numpy and/or pandas in your search.

_______

## Why do we care about numpy?
1. Our code is faster
3. Our code is (often) more readable
2. Our code is (almost always) more intuitive

#### For example:  Implementing a simple  [random walk](https://en.wikipedia.org/wiki/Random_walk)
i.e. at each step, move either one place forward or one place backward

In [ ]:
# python implementation - requires for loop
import random

def random_walk(n):
    '''Randomly walk n steps'''
    position = 0
    walk = [position]
    for i in range(n):
        position += random.choice([-1, 1])
        walk.append(position)
    return walk

%timeit random_walk(1000)

In [ ]:
# numpy implementation - no for loop, ~100x faster, more readable
def random_walk(n):
    '''Randomly walk n steps'''
    steps = np.random.choice([-1,1], size=n) 
    return np.cumsum(steps)

%timeit random_walk(1000)

The idea of removing `for` loops in favour of creating and manipulating whole arrays at a time is central to numerical computing in python, and most of what follows focuses on it. You can think of this as moving from thinking in terms of scalars to instead thinking in terms of arrays. Which is also central to TensorFlow - and if you believe the hype, that's the ticket to getting people to react to you like: <img src=http://i0.kym-cdn.com/photos/images/original/000/515/629/9bd.gif alt="OMG cat">

But while a particular library can give us some extremely useful tools, it's far from the whole story on writing good clean and clear code, or even fast code.

## How do I make my code fast?
There are two main ways to speed up your computations:
-  __move computation or memory allocation outside a `for` loop__
-  __compute less, or better__. 

Python can often be made faster if you don't settle for the first implementation that comes to mind. "Make use of numpy" is the theme, but the more general lesson is "think about the implementation!"

#### For example, how can we most efficiently solve this:
> Compute all 4-digit combinations which sum to 10. -- e.g. (0,0,0,10), (1,0,0,9), ...

In [ ]:
def naive_brute_force():
    '''Compute all 4-digit combinations which sum to 10'''
    # 11^4 = 14641 iterations & tests
    Z = []
    for i in range(11):
        for j in range(11):
            for k in range(11):
                for l in range(11):
                    if i+j+k+l == 10:
                        Z.append((i,j,k,l))
    return Z

%timeit naive_brute_force()

The final `for` loop and `if` statement are redundant, so we can be more efficent and get an order of magnitude speed-up.

In [ ]:
def compute_less():
    '''Compute all 4-digit combinations which sum to 10'''
    Z = []
    for a in range(11):
        for b in range(11 - a):
            for c in range(11 - a - b):
                Z.append((a, b, c, (10 - a - b - c)))
    return Z

%timeit compute_less()

**Readability counts**. So for slightly neater code, we might prefer to put this in a list comprehension (which also gains us some negligible speed-up)

In [ ]:
def neat_list_comprehension():
    '''Compute all 4-digit combinations which sum to 10'''
    return [(a, b, c, (10 - a - b - c))
            for a in range(11)
            for b in range(11 - a)
            for c in range(11 - a - b)]

%timeit neat_list_comprehension()

**Think about the use case.** Here we're doing probably answering a single simple question ("How many 4-digit combinations sum to 10?"), so want to be able to call `len` on the list we calculate. 

But what if we don't need to use values immediately, or if we expected our list to be much, much longer? To take an extreme case - what if we were calculating the Fibonacci sequence? Then we probably just want to be able to iterate through the values, and can create a **generator instead of a list**. This gives a further couple orders of magnitude speed up, because we're not actually creating the values (i.e. it's not doing the same computation as the functions above). 

In [ ]:
def generator_comprehension():
    '''Compute all 4-digit combinations which sum to 10'''
    return ((a, b, c, (10 - a - b - c))
            for a in range(11)
            for b in range(11 - a)
            for c in range(11 - a - b))

%timeit generator_comprehension()

> Aside: Remember generators return an _iterator_ which returns a stream of values (they don't return values themselves, so are more like function definitions). For example:

In [ ]:
# define a trivial generator
def generate_ints(n):
    for i in range(n):
        yield i

# create a generator
gen = generate_ints(4)
gen

In [ ]:
# re-run this cell multiple times
next(gen)

#### A view or a copy - what's the difference?
Generators might be an example of "computing less" or "computing better", depending on how you look at it. Another example that fits into a simpler place is the idea of creating views, rather than copies (when views are okay, and that's sometimes a tricky question).

In [ ]:
# creating a copy of a big array is nearly as
# costly as simple numerical operations on them
a = np.zeros(10**6)
%timeit a.copy()
%timeit a + 1

__Views__ means that the data of both objects is shared. You can create views by selecting a slice of the original array, or also by changing the dtype. I won't talk more about this here, but looks out for 'creates a view' and 'creates a copy' when reading the documentation of functions. And there's more [info here](http://scipy-cookbook.readthedocs.io/items/ViewsVsCopies.html)

## What makes numpy fast?

You might wonder how it's possible for an extension to a language, like the numpy extension of python, can make computations faster - how, for example, it can move to an array-first approach. If you're not wondering that - move right on to the actual coding!

The key is that a numpy array is stored in a *contiguous block of memory*. This [locality in memory](https://en.wikipedia.org/wiki/Locality_of_reference)
allows for __faster accessing and for specialised implementations__ of method, many of which are actually executed in C, not python.

In fact, items are (often) stored as C arrays, which are statically typed. Items in a numpy array are *the same data type*, meaning we (often) __avoid the cost of per-element dynamic type checking__.

The items in an array can be accessed using an *indexing scheme*. The indexing scheme is by shape and data type -- exactly what is needed in defining a new array.

ndarray = block of memory (raw data) + indexing scheme (how to locate an element) + data type descriptor (how to interpret an element)

_(The complete story involves a few more complications)_

In [ ]:
new_array = np.arange(12).reshape(4,3).astype(np.int8)
print(new_array)

`new_array` is stored as 12 bytes, one after the other (a contiguous block of memory). 

In [ ]:
print('new_array has')
print('shape:', new_array.shape)
print('dimension:', new_array.ndim) # equivalent to len(new_array.shape)
print('item size:', new_array.itemsize) # we specified 8-bit integers (so each is 1-byte)

#### Traversing a numpy array
The **strides** of an array tell us how many bytes we have to skip in memory to move to the next position along a certain axis. 

For example, in our (4, 3) array of 8-bit integers we have to skip 1 byte (1 value) to move to the next column, but 3 bytes (3 values) to get to the same position in the next row. As such, the strides for the `new_array` are (3, 1).

In [ ]:
strides = (new_array.shape[1] * new_array.itemsize, new_array.itemsize)
assert strides == new_array.strides
print(strides)

Let's see this in action

In [ ]:
c = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]], dtype=np.int16) # note ints are 16-bit == 2-bytes
c.data.tobytes()

At which byte does `c[2, 1]` begin?

In [ ]:
# what's the stride length for each dimension?
print(c.strides)

In [ ]:
# get to [2, 1]
c.data.tobytes()[6*2 + 2*1] == c[2, 1]

___

## The majority of this notebook provides challenges for you. 
They come in two parts:
1. A series of examples and coding challenges to build your skills in numpy
2. A series of problems that require you to [vectorise](https://en.wikipedia.org/wiki/Array_programming) : re-implement functions to apply operations to entire subsets of your data, rather than item by item.

The accompanying notebook `Pandas.ipynb` covers
1. Querying and merging data : Using pandas as an in-memory database.
2. Cleaning and transforming data : Showing off the wonders of pandas.

You can also ctrl+f for 'Task' to find all the programming challenges.

# 1. Hello Numpy World 
Let's see what I'm on about. 

We'll cover:
    
    i. Creating data arrays
    ii. Indexing 
    iii. Reshaping arrays
    iv. Broadcasting scalars and arrays to different sizes
    v. Matrix operations


## i. Creating data

In [ ]:
np.zeros(5)

In [ ]:
np.ones(5)

In [ ]:
np.array([1,0,0,1,0]) # note: argument is a list [...]

In [ ]:
# evenly spaced
np.linspace(0, 1, num=5)

#### Task: create some data on a log scale

In [ ]:
## your code here

In [ ]:
r = np.random.randint(0, 6, size=(4, 3))
r

In [ ]:
np.zeros_like(r)

#### Task: When would `np.zero ` be useful in machine learning. When might we prefer `np.zero_like`?
Relatedly, why might over-writing zeros with new values be preferable to just appending new values to an empty list

In [ ]:
## actually think. maybe google to find cases where it's used

Random data:

In [ ]:
# Gaussian
np.random.randn(4)      

#### Task: Try setting the seed before creating an array with random values. (Generate data from a different distribution. Try to figure out what `np.random.seed` does.)

In [ ]:
np.random.seed(1)  
## your code here

## ii. Access data by indexing

In [ ]:
a = np.arange(9).reshape(3,3)
a

In [ ]:
a[2,2] # item by index

In [ ]:
a[1] # row by index

In [ ]:
a[:,2] # column by index

In [ ]:
a[1:, :2] # what is this doing?

#### Task: Using a single index, of the form `a[:,:]`, access the 4 corner values of our matrix `a` 

In [ ]:
## your code here

#### Task:  Select the even numbers from the (5, 5) array defined below

In [ ]:
a = np.arange(25).reshape(5, 5)
## your code here

_Indexing_
![](../input/pictures/numpy_indexing.png)

#### Task: return the 2nd, 3rd and 5th item in every other row in `P`, starting with row 0
Hint: look at np.ix_

In [ ]:
P = np.random.rand(5, 5)
## your code here

#### Task: Generate a 10 x 3 array of random numbers in range [0,1]. For each row, pick the number closest to 0.5.

In [30]:
## your code

#### Task: You're given two matrices of the same shape. Select values from the first if the values in the second are positive. 

In [ ]:
first = np.random.randint(10, size=(10,10))
second = np.random.randn(100).reshape(10,10) - 0.3
## your code

## iii. Reshaping

In [ ]:
z = np.zeros(6)
print(z.shape)
print(z)

In [ ]:
z.reshape(len(z), 1)

In [ ]:
z[3] = 1
z = z.reshape(3,2)
z

In [ ]:
z

In [ ]:
z.itemset((2,1), 8) # set in-place
z = z.reshape(2, -1) # unspecified (-1) dimension inferred
z

In [ ]:
# transpose 
z.T

In [ ]:
print(z.ravel()) # view
print(z.flatten()) # copy

#### Task: Create a 3-dimensional array, then use indexing to return the first dimension

In [ ]:
## your code here

#### Task: split `arrays` into 3 arrays with shape (10, 4)

In [ ]:
arrays = np.stack([np.random.randn(3, 4)
                   for _ in range(10)], axis=0)
print(arrays.shape)

## your code here

## iv. Broadcasting
On numpy arrays operations, like `+`, `-`, `*`,  are elementwise. It’s possible to do __operations on arrays of different sizes__ when numpy can transform them to be the same size (known as "broadcasting").

In [ ]:
np.ones(5) * 2 

You could get the same result in python like `[2] * 5` but that's not quite the same operation... 
#### Task: In plain python, if you have data in a list like `[1, 1, 1, 1, 1]` how would you multiply the values by 2?

In [ ]:
l =  [1, 1, 1, 1, 1]
## your pure python code here

#### Task: In plain python, if you have data in a list like `[2, 2, 2, 2, 2]` how would you multiply the values elementwise by a second list like `[3, 6, 12, 24, 48]`?

In [ ]:
## your code here

In [ ]:
Z = np.arange(9).reshape(3,3)
Z

In [ ]:
Z + 1

1 was 'broadcast' into the same shape as Z, i.e. `np.ones(shape=(3,3))`

In [ ]:
np.alltrue(Z + 1 == Z + np.ones((3,3)))

In [ ]:
# let's make the values in a row all the same by
# adding this [2, 1, 0] to the corresponding columns
this = np.arange(3)[::-1]
print(this)
Z + this

#### Task: Create a (10, 9) 2-d array where values on the same row all have the same value
hint: look at `np.tile` and `np.repeat` and `np.broadcast_to`

In [ ]:
## your code here

#### Task: Make use of broadcasting to add 3 to first row, 2 to the second row and 1 to the third row of Q

In [ ]:
Q = np.arange(9).reshape(3,3)
## your single line of code here

_Broadcasting_
![](../input/pictures/numpy_broadcasting.png)

### v. Matrix operations

So far we've always worked with N-dimensional arrays (type `numpy.ndarray`), which can be created with `np.array`. Numpy also has a `np.matrix` method which creates strictly 2-dimensional matrices (with type `numpy.matrixlib.defmatrix.matrix`).

In [ ]:
np.matrix?

In [ ]:
M = np.matrix('1 2; 3 4')
N = np.matrix([[4, 3], [2, 1]])

Matrices have different behaviour to arrays.

For example, remember that operations on arrays are usually elementwise. Which often leads to convenient behaviour, for example:

In [ ]:
A = np.array([[1, 2], [3, 4]])
A * 2

But, some people can find elementwise behaviour unintuitive. For example:

In [ ]:
B = np.array([[4, 3], [2, 1]])

# will return elementwise mutiplication
A * B

When multiplying 2x2 arrays together, one might expect _matrix multiplication_ (i.e. the dot product). This is exactly what you get when multiplying numpy matrices

In [ ]:
N * M

For arrays we need to explicitly use `np.dot`

In [ ]:
A.dot(B)

But, different behaviour like this is pretty much the main benefit of using the `matrix` type. It's also the main downside. Writing a program that uses both matrices and arrays makes your life difficult because you have to keep track of what type of object your variables are, lest multiplication (or `ravel` or whatever) returns something you don't expect.

In contrast, sticking solely with `ndarrays`, __`arrays` can do everything `matrix` objects can do, and more__, except with slightly different functions/notation. In fact, this means you only need to know one set of functions and there behaviour on arrays.

If you are willing to give up the visual appeal of numpy matrix product notation, then numpy arrays are definitely the way to go. 

**Let's focus on common matrix operations, using arrays**

#### Task: Find a vector `y` such that the dot product of `W` and `y` = 0

In [ ]:
W = np.arange(1, 10).reshape(3,3).T
## your code here

#### Task: Determine if X is invertible

In [ ]:
X = np.random.randint(99, size=(12, 12))
## your code

#### Task: You want to save a large grey-scale image, 300x300 pixels with values 0.0 to 1.0 (bigger is darker). Fortunately, the image is not very interesting - it's just a black sqare. How would you save the image more efficiently, so that it can easily be recreated? Define an array and save it as `.npy`.
Hint: 1. Saving `np.ones((300, 300))` is not sufficiently efficient. 2. Look at `np.save`


In [ ]:
## your code here

#### Task: write a function that takes your saved filename as input and returns your very interesting plain black square (either as an array or plotted).

In [ ]:
## your code here

#### Task: 
You started saving slightly more interesting images: national flags. But you're still as comitted to decomposing them in ways that allow you to save them more efficiently. To allay worries that they're not being recreated properly: write a function that takes two ndarrays as input and tells us whether or not they recreate a scandinavian flag (as defined below).

In [ ]:
sweden = np.array([[6, 6, 3, 6, 6, 6],
                   [6, 6, 3, 6, 6, 6],
                   [3, 3, 3, 3, 3, 3],
                   [6, 6, 3, 6, 6, 6],
                   [6, 6, 3, 6, 6, 6]])

denmark = np.array([[0, 0, 1, 0, 0, 0],
                    [0, 0, 1, 0, 0, 0],
                    [1, 1, 1, 1, 1, 1],
                    [0, 0, 1, 0, 0, 0],
                    [0, 0, 1, 0, 0, 0]])

norway = np.array([[0, 1, 2, 1, 0, 0],
                   [1, 1, 2, 1, 1, 1],
                   [2, 2, 2, 2, 2, 2],
                   [1, 1, 2, 1, 1, 1],
                   [0, 1, 2, 1, 0, 0]])

In [ ]:
## your code here

#### Task: Using  `np.linalg.svd`, find and graph the singular values of A and B. 

In [ ]:
A = np.random.uniform(0, 1, size=(20, 40))
B = np.random.randn(20, 40)

Linear algebra methods can be applied to several matrices at once, if stacked into the same array.

For an input array `a`, if `a.shape == (N, M, M)` it is interpreted as a “stack” of `N` matrices, each of size `M-by-M`. Similar specification applies to return values, e.g. `np.det(a).shape == (N,)`.

_______

# 2. Vectorising operations
Strighting out operations to avoid loops. In other words, let's apply operations to entire subsets of data, rather than item by item.

#### Task: Write an `add` function in plain python that takes two lists as input and returns their elementwise sum

In [ ]:
def add(first, second):
    '''Elementwise sum'''
    ## your code here

#### Task: Write an `add` function using numpy

In [ ]:
def vectorised_add(first, second):
    '''Elementwise sum'''
    ## your code here

#### Task: Compute the euclidean distance between all points on a grid of evenly spaced values
- Create an array of 1000 evenly spaced values
- Create a grids of all possible (x, y) values from the array (look at `np.meshgrid`)
- Compute a matrix `d` containing euclidean_distance(x, y) for each (x, y) pair

To check your answer, we provide code for plotting `d`.

In [ ]:
## your code here 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(d, cmap=plt.cm.gray)
plt.colorbar()
plt.show()

## Vectorising functions

### Task: Applying this in machine learning 
In supervised machine learning, we use labelled data to find the parameters of a function that accurately maps the raw data to the corresponding labels. We 'learn' the parameters by quantifying how well our current model is doing and using some optimization criterion to update the parameters in a direction that improves predictions.

Therefore, a common machine learning task is minimising a function which quantifies the agreement between our model's predictions and the ground truth 
(call this function a [_loss function_](https://en.wikipedia.org/wiki/Loss_function) or _cost function_).

Of course when training a model, we want to be able to get fast feedback on how well we're doing, so it's important to calculate the loss function efficiently. Below, we give you a dataset of images belonging to 10 categories, a function that maps the raw data to class scores and a (inefficient, non-vectorised) loss function.
#### You need to speed the learning up, by implementing a vectorised version of the loss function.

- **Data** : 60,000 tiny images belonging to 10 different categories. _For example:_
![](../input/pictures/cifar10.jpeg)

In [ ]:
# you don't have to read this
import pickle
def unpickle(file):
    '''Unpickle data from binary'''
    with open(file, 'rb') as f:
        return pickle.load(f, encoding='bytes')

def load_cifar10_data(folder='../input/Cifar-10'):
    '''Load train and test sets of CIFAR 10 data.
    
    Parameters
    ----------
    folder : string, optional (default = 'Cifar-10-data')
        Path to folder containing data.
        
    Returns
    -------
    X_train : numpy array, shape (50000, 3072)
        Matrix of flattened 32x32 pixel RGB images.
    y_train : numpy array, shape (50000,)
        Vector of corresponding labels for images.
    X_test : numpy array, shape (10000, 3072)
    y_test : numpy array, shape (10000,)
    '''
    trainingset = {}
    for i in np.arange(1,6):
        trainingset[i] = unpickle('{}/data_batch_{}'.format(folder, i))
    # 'b' prefix required for bytes literals in python 3
    X_train = np.vstack([trainingset[i][b'data'] for i in np.arange(1,6)])
    y_train = np.hstack([trainingset[i][b'labels'] for i in np.arange(1,6)])

    testset = unpickle('{}/test_batch'.format(folder))
    X_test = testset[b'data']
    # testset labels are a list. Make np.array for consistency.
    y_test = np.array(testset[b'labels'])
    
    return X_train, y_train, X_test, y_test

- **Score function** : We'll use a simple linear model $$ s = f(x, W) = Wx $$ (Ignoring the usual bias term $b$. If you wish to include it, extend $W$ to have an extra column and each image vector $x$ with a $1$ that multiplies the new bias column of $W$).
    - Specifically, $x$ is an image, so a 1-d array of 32x32x3 = 3072 pixel values
    - We want to assign $x$ a score for each of the 10 possible labels, so $s$ will be a vector of length 10, and thus $W$ will have shape (10, 3072)
![](../input/pictures/linearClassifier.jpg)

In [ ]:
def score(x, W): 
    return np.dot(W, x)

- **Loss function** : We'll use $ L = max(0,-) $ sometimes called the _hinge loss_.
    - Specifically the loss for a single image $x_i$ is $$ L_i = \sum_{j \neq y_i} max(0, s_j - s_{y_i} + \Delta ) $$
        - $j$ are the 10 possible classes,
        - $y_i$ is the correct class for image $x_i$, and
        - $s$ is the vector of scores assigned by the score function (so $s_j$ is the score assigned to the $j^{th}$ class)
    - i.e. the loss $L_i$ for a single image $x_i$ equals zero when the score function assigns the correct class $y_i$ a score that is at least delta greater than the sore assigned to any other class, for some hyperparameter delta.
![](../input/pictures/hingeLoss.jpg)

In [ ]:
# non-vectorised
def L_i(x_i, y_i, W, delta=1.):
    '''Calculate hinge loss for single image.
    
    Very inefficent. Takes only a single 
    image as input and loops over class.
    
    Parameters
    ----------
    x_i : numpy array
        flattened input image pixels, shape (N,)
    y_i : int
        index of the true class label (in Cifar-10
        this is the same as the true class label).
    W : numpy array
        weight matrix, shape (K, N)
    delta : float, optional (default = 1.)
    '''    
    scores = score(x_i, W)
    loss_i = 0.0
    for j in np.arange(scores.shape[0]):
        if j != y_i:
            loss_i += max(0, scores[j] - scores[y_i] + delta)
    return loss_i

Let's check the function runs.

In [ ]:
# hand set a score vector 
x = np.array([13, -7, 11])
W = np.eye(3) # 3x3 identity matrix

# arbitrarily say the first entry is the score for the correct class
y = 0
# since the second entry (-7) is negative, the loss should be 11 - 13 + delta
loss = L_i(x, y, W, delta=10.)
print('loss =', loss)

See how fast it is on a proper dataset of 50,000 images.

In [ ]:
# not going to use the test set
X_train, y_train, _, _ = load_cifar10_data()

# random weights
W = np.random.randn(30720).reshape(10, 3072)

In [ ]:
%%timeit
total_loss = 0
for x_i, y_i in zip(X_train, y_train):
    total_loss += L_i(x_i, y_i, W)
print(total_loss)

#### Task: speed the learning up, by implementing a vectorised version of the loss function.

In [ ]:
## EXERCISE
def L(X, y, W):
    '''Calculate hinge loss more efficiently.'''
    
    #### YOUR 
    #### CODE
    #### HERE
    
    return loss

Hints:
- You could first vectorise the calculation of the loss for a single image (which would still require us to loop over the entire training set, but not every pixel). Then, figure out how to modify that function to vectorise the calculation of the loss for an entire training set.


- Remember to exclude the score assigned to the correct class when calculating the loss. You're not using a `for` loop, so you're not going to use an `if` statement either... how else can numpy be used to select out an element instead?


- Look at the various `max` functions available in numpy.


- You might want to rewrite the `score` function

Note: we've only given you a small dataset (50,000 32x32px images), which is probably not big enough to see the speed benefits of vectorising. The ImageNet dataset which is standardly used for training and benchmarking image classifiers is more like 15 million higher-quality images.

Aside:
- Numpy also has a `np.vectorize` method you can investigate (I think it's for tidiness and convenience, rather than speed, but I've not used it yet - please teach me if it's useful).

#### Task: 
You know that once you actually train your model and learn seemingly good parameters for your score function, you're going to want to benchmark the predictions against another model. 

You decide to pick the simplest possible model: a k-nearest neighbour classifier, which simply calculates the distance between the input image and all other images, based on elementwise pixel distances, and labels the input with the same class label as the nearest image (or taking a majority vote of the k nearest images).

#### Write the predict method for the KNN classifier 
Again, avoid looping over the pixels (ideally avoid looping over every image too).
#### You could also implement a simple distance function (e.g. L1 or L2 norm)

In [ ]:
class KNearestNeighbours(object):
    
    def __init__(self):
        pass
    
    def train(self, X, y):
        '''Load the data into memory.
        
        Parameters
        ----------
        X : array_like, shape (N, D)
            Matrix of N training examples.
        
        y : array_like, shape (N,)
            Vector of N training labels.
        '''
        self.X_train = np.array(X)
        self.y_train = np.array(y)
        
    def predict(self, X, distance, k=1):
        '''Predict class labels for X.
        
        Parameters
        ----------
        X : numpy array, shape (M, D)
            Matrix of M examples to label.
            
        distance : function
            Method for calculating distance
            between Xs and training data.
            
        k : int, optional (default = 1)
            Number of neighbours to consider.            
        
        Returns
        -------
        y_pred : numpy array, shape (M,)
            Vector of predicted labels for X.
        '''        

        ### your
        ### code
        ### here
        
        return y_pred

_______

#### Whenever you think of writing your own function:
 - First, check if numpy or pandas can do it for you (in the cases above, we could actually have looked at sklearn or scipy too).
 - Then, if not, make use of numpy and pandas for writing a vectorised function
 
Caveats:
1. Debugging is difficult and inevitable. It's also harder than writing code in the first place. So, if you’re as clever as you can be when you write it, how will you ever debug it? Make use of vectorisation to write faster, cleaner code, not to be a smart arse.
2. Maintenance is also inevitable. Which means readability counts. When vecotrising code, that mean:
    - If the implementation is hard to explain, it's a bad idea.
    - If the implementation is easy to explain, it _may_ be a good idea.

## Credit
- Some of this notebook edits and builds on resoures found in the open-source book [From Python to Numpy](http://www.labri.fr/perso/nrougier/from-python-to-numpy/)
- Other parts make use of the resurces found in [scipy lecture notes](http://www.scipy-lectures.org/)
- The problem of vectorising a hinge loss function and implementing KNN is based on [CS231n](http://cs231n.github.io/).
- The idea of flags as matrices comes from somewhere in a Gilbert Strang book.
- Thanks to Andrew Crozier for suggestions that made major imporovements

Any errors and inefficiency in the code were introduce by Nick. Feedback saying which bit were useful and which bits were not to @nick on slack. Genuine rewards (tasty snacks) for people who suggest a new exercise that gets included in v2.0 of this notebook.

In [ ]:
# little pythonic treat
import this

Copyright © ASI 2017 All rights reserved